In [13]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [14]:
from torch.utils.data import DataLoader, Dataset

In [15]:
import pandas as pd

In [16]:
from torchvision.io import read_image

In [17]:
from pathlib import Path

In [37]:
labels.comment_number.loc[19999][3:]

'  A dog runs across the grass .'

In [38]:
class ImageCaptioningDataset(Dataset):
    def __init__(self,caption_file,image_dir,image_transform=None):
        self.image_transform = image_transform
        self.image_dir = Path(image_dir)
        captions_csv = pd.read_csv(caption_file, sep="|")
        self.captions_csv = captions_csv.rename(lambda x: x.strip(), axis=1)
        self.captions_csv['comment'] = self.captions_csv['comment'].str.lower()
        self.captions_csv.comment.loc[19999] = self.captions_csv.comment_number.loc[19999][3:]
        self.captions_csv.comment_number.loc[19999] = 4
        display(self.captions_csv['comment'])
        self.tokenizer = get_tokenizer('spacy', language='en_core_web_sm',)
        self.list_of_tokens = self.captions_csv['comment'].apply(self.tokenizer)
        self.vocab = build_vocab_from_iterator(self.list_of_tokens)
    def __len__(self):
        return self.captions_csv.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.image_dir / Path(self.captions_csv.loc[idx,'image_name'])
        image = read_image(str(image_path))
        if self.image_transform:
            image = self.image_transform(image)
        return image, self.vocab(self.list_of_tokens[idx])

In [39]:
dataset = ImageCaptioningDataset('./flickr30k_images/results.csv', './flickr30k_images/flickr30k_images/',)

0          two young guys with shaggy hair look at their...
1          two young , white males are outside near many...
2          two men in green shirts are standing in a yard .
3              a man in a blue shirt standing in a garden .
4                   two friends enjoy time spent together .
                                ...                        
158910     a man in shorts and a hawaiian shirt leans ov...
158911     a young man hanging over the side of a boat ,...
158912     a man is leaning off of the side of a blue an...
158913     a man riding a small boat in a harbor , with ...
158914     a man on a moored blue and white boat with hi...
Name: comment, Length: 158915, dtype: object